In [1]:
import pandas as pd
import numpy as np
%matplotlib inline 
import matplotlib.pyplot as plt
import MetaTrader5 as mt5
from account_credentials import LOGIN, PASSWORD, SERVER
from datetime import datetime, timedelta

In [2]:
is_initialized = mt5.initialize()
print('initialize: ', is_initialized)

is_logged_in = mt5.login(LOGIN, PASSWORD, SERVER)
print('logged in: ', is_logged_in)
print('\n')
account_info = mt5.account_info()
print(datetime.now(),
    '| Login: ', account_info.login,
    '| Balance: ', account_info.balance,
    '| Equity: ' , account_info.equity)

initialize:  True
logged in:  True


2023-04-20 06:16:57.747544 | Login:  114123121 | Balance:  538.36 | Equity:  532.1


In [3]:
import MetaTrader5 as mt5
from datetime import datetime, timedelta

# connect to MetaTrader 5
mt5.initialize()

# set the time range for the history
start_time = datetime.now() - timedelta(days=100)
end_time = datetime.now()

# retrieve the history
history = mt5.history_deals_get(start_time, end_time)

# Create an empty dataframe
df = pd.DataFrame(columns=['Time','Deal ID', 'Order Ticket', 'Type', 'Volume', 'Price', 'Profit', 'Comment', 'Magic'])

# Iterate through the trade history and create a list of dataframes
dfs = []
for deal in history:
    row = {'Time': pd.Timestamp(deal.time, unit='s'),
           'Deal ID': deal.order,
           'Order Ticket': deal.ticket,
           'Type': 'Buy' if deal.type == 0 else 'Sell',
           'Volume': deal.volume,
           'Price': deal.price,
           'Profit': deal.profit,
           'Comment': deal.comment,
           'Magic': deal.magic}
    dfs.append(pd.DataFrame(row, index=[0]))

# Concatenate all dataframes into a single dataframe
df = pd.concat(dfs, ignore_index=True)

# Print the resulting dataframe
print(df)



                   Time    Deal ID  Order Ticket  Type  Volume     Price   
0   2023-03-23 01:46:47          0     143696145  Sell    0.00     0.000  \
1   2023-03-28 02:09:05  266948417     145348938   Buy    0.01  1960.969   
2   2023-03-28 02:29:00  266953667     145353163   Buy    0.01  1960.427   
3   2023-03-28 03:03:56  266962782     145360604   Buy    0.02  1961.108   
4   2023-03-28 04:18:59  266982644     145377761  Sell    0.02  1958.718   
..                  ...        ...           ...   ...     ...       ...   
249 2023-04-19 17:00:00  278102545     152930008   Buy    0.01  1996.139   
250 2023-04-19 17:00:01  278102566     152930019   Buy    0.01  1996.281   
251 2023-04-19 17:00:02  278102573     152930023   Buy    0.01  1996.381   
252 2023-04-19 18:00:00  278118473     152940995  Sell    0.01  1994.012   
253 2023-04-19 20:00:00  278139142     152952691  Sell    0.01  1994.288   

     Profit                       Comment   Magic  
0    500.00  D-trial-USD-INT-98b286

In [22]:
df_adjust = df[(df['Profit'] != 0) & (abs(df['Profit']) > 3) & (abs(df['Profit']) != 500)]
df_adjust.reset_index(inplace=True)
df_adjust.to_csv('history_price.csv')
df_adjust

,index,Time,Deal ID,Order Ticket,Type,Volume,Price,Profit,Comment,Magic
0,4,2023-03-28 04:18:59,266982644,145377761,Sell,0.02,1958.718,-4.78,,0
1,56,2023-03-29 15:08:09,267851674,146078214,Sell,0.01,1966.148,-3.12,[sl 1966.148],123999
2,58,2023-03-29 17:41:33,267915037,146134790,Sell,0.01,1968.253,3.12,[tp 1968.253],123999
3,60,2023-03-29 19:01:04,267933799,146151811,Sell,0.01,1966.052,-3.13,[sl 1966.052],123999
4,76,2023-03-30 16:00:46,268440990,146546900,Sell,0.01,1975.363,3.12,[tp 1975.363],123999
...,...,...,...,...,...,...,...,...,...,...
83,240,2023-04-19 13:45:49,277992382,152843271,Sell,0.01,1991.212,3.12,[tp 1991.212],123994
84,244,2023-04-19 14:06:25,278008087,152856143,Sell,0.01,1995.217,4.43,[tp 1993.919],123992
85,245,2023-04-19 14:06:25,278008088,152856148,Sell,0.01,1995.217,4.36,[tp 1993.985],123993
86,246,2023-04-19 14:06:25,278008126,152856205,Sell,0.01,1995.232,4.28,[tp 1994.076],123999


In [79]:
print("------MODEL--------")
# TP and SL should be 3.13
TPSL_base = 3.13

print("All Samples",len(df_adjust[df_adjust["Magic"] == 123999]))
m1_df = df_adjust[df_adjust["Magic"] == 123999]
Profit = sum(m1_df["Profit"])
print(f"Profit ${Profit:.2f}")
Accuracy = (m1_df["Profit"] > 0).sum() / len(m1_df)
print(f"Accuracy {Accuracy*100:.2f}%")


Lost_standard = ((m1_df["Profit"] < 0).sum() * TPSL_base) #Positive Value
Lost_actual = (sum(m1_df[m1_df["Profit"] < 0]["Profit"])) # Negative Value
Lost_error = Lost_standard + Lost_actual # if result = positive then Lost_standard > Lost_actual
print(f"Lost Error ${Lost_error:.2f}")

Win_standard = ((m1_df["Profit"] > 0).sum() * TPSL_base) #Positive Value
Win_actual = (sum(m1_df[m1_df["Profit"] > 0]["Profit"])) # Positive Value
Win_error = Win_actual - Win_standard # if result = positive then the actual win is higher than standard win
print(f"Win Error ${Win_error:.2f}")



------MODEL--------
All Samples 37
Profit $23.51
Accuracy 59.46%
Lost Error $0.10
Win Error $1.50


In [82]:
#type SEll >>> คือ buy ที่ขายออก
#type Buy >>> คือ sell ที่ซื้อออก
def get_summary(df_adjust, Magic, TPSL_base, Type):
    if Type == 'Buy':
        Order_type = 'Sell'
    if Type == 'Sell':
        Order_type = 'Buy'
    buy = df_adjust[(df_adjust["Magic"] == Magic) & (df_adjust["Type"] == Order_type)]
    buy_profit = buy["Profit"]
    Profit = round(sum(buy_profit), 2)
    Accuracy = round((buy["Profit"] > 0).sum() / len(buy["Profit"]), 2)
    standard_buy = ((buy["Profit"] > 0).sum() * TPSL_base) - ((buy["Profit"] < 0).sum() * TPSL_base)
    error_buy = round(Profit - standard_buy, 2)
    return Order_type, Profit, Accuracy, error_buy

Order_type, Profit, Accuracy_buy, error_buy = get_summary(df_adjust, 123999, 3.13, "Buy")
print(f"Profit_buy: {Profit}")
print(f"Accuracy {Accuracy*100:.2f}%")
print(f"Error buy ${error_buy:.2f}")

Profit_buy: 23.51
Accuracy 59.46%
Error buy $1.60


In [75]:
print("------MODEL v2--------")
# TP and SL should be 3.13
TPSL_base = 3.13

print("All Samples",len(df_adjust[df_adjust["Magic"] == 123992]))
print()
b2_buy = df_adjust[(df_adjust["Magic"] == 123992) & (df_adjust["Type"] == "Buy")]
b2_buy_profit = b2_buy["Profit"]
Profit_buy = round(sum(b2_buy_profit),2)
print("Buy samples: ",len(b2_buy_profit))
print("Sum profit b2 buy",Profit_buy)
Accuracy_buy = round((b2_buy["Profit"] > 0).sum() / len(b2_buy["Profit"]),2)
print(f"Accuracy {Accuracy_buy*100:.2f}%")
standard_buy = ((b2_buy["Profit"] > 0).sum()*TPSL_base)-((b2_buy["Profit"] < 0).sum()*TPSL_base)
error_buy = round(Profit_buy - standard_buy,2)
print(f"Error buy ${error_buy:.2f}")

print()

b2_sell = df_adjust[(df_adjust["Magic"] == 123992) & (df_adjust["Type"] == "Sell")]
b2_sell_profit = b2_sell["Profit"]
Profit_sell = round(sum(b2_sell_profit),2)
print("Sell samples: ",len(b2_sell_profit))
print("Sum profit b2 sell", Profit_sell)
Accuracy_sell = round((b2_sell["Profit"] > 0).sum() / len(b2_sell["Profit"]),2)
print(f"Accuracy {Accuracy_sell*100:.2f}%")
standard_sell = ((b2_sell["Profit"] > 0).sum()*TPSL_base)-((b2_sell["Profit"] < 0).sum()*TPSL_base)
error_sell = round(Profit_sell - standard_sell,2)
print("Error sell",error_sell)

print()

print("Overall MODEL v2")
Profit = Profit_buy + Profit_sell
print("Profit",Profit)
Accuracy = (Accuracy_buy + Accuracy_sell) / 2
print(f"Accuracy {Accuracy*100:.2f}%")
Error = round(error_sell + error_buy,2)
print("Error",Error)

------MODEL v2--------
All Samples 39

Buy samples:  25
Sum profit b2 buy -4.05
Accuracy 52.00%
Error buy $-7.18

Sell samples:  14
Sum profit b2 sell 20.41
Accuracy 71.00%
Error sell 1.63

Overall MODEL v2
Profit 16.36
Accuracy 61.50%
Error -5.55


Profit_buy: -4.05
Accuracy_buy: 0.52
error_buy: -7.18


In [38]:
print("------MODEL v2--------")

# df_adjust Magic TPSL_base
#buy_profit # Profit_buy Accuracy error_buy

# TP and SL should be 3.13
TPSL_base = 3.13

print("All Samples",len(df_adjust[df_adjust["Magic"] == 123993]))
print()
buy = df_adjust[(df_adjust["Magic"] == 123993) & (df_adjust["Type"] == "Buy")]
buy_profit = buy["Profit"]
Profit_buy = round(sum(buy_profit),2)
print("Buy samples: ",len(buy_profit))
print("Sum profit b2 buy",Profit_buy)
Accuracy = round((buy["Profit"] > 0).sum() / len(buy["Profit"]),2)
print(f"Accuracy {Accuracy*100:.2f}%")
standard_buy = ((buy["Profit"] > 0).sum()*TPSL_base)-((buy["Profit"] < 0).sum()*TPSL_base)
error_buy = round(Profit_buy - standard_buy,2)
print("Error buy",error_buy)


print()

b2_sell = df_adjust[(df_adjust["Magic"] == 123993) & (df_adjust["Type"] == "Sell")]
b2_sell_profit = b2_sell["Profit"]
Profit_sell = round(sum(b2_sell_profit),2)
print("Sell samples: ",len(b2_sell_profit))
print("Sum profit b2 sell", Profit_sell)
Accuracy_sell = round((b2_sell["Profit"] > 0).sum() / len(b2_sell["Profit"]),2)
print(f"Accuracy {Accuracy_sell*100:.2f}%")
standard_sell = ((b2_sell["Profit"] > 0).sum()*TPSL_base)-((b2_sell["Profit"] < 0).sum()*TPSL_base)
error_sell = round(Profit_sell - standard_sell,2)
print("Error sell",error_sell)

print()

print("Overall MODEL v2")
Profit = Profit_buy + Profit_sell
print("Profit",Profit)
Accuracy = (Accuracy_buy + Accuracy_sell) / 2
print(f"Accuracy {Accuracy*100:.2f}%")
Error = round(error_sell + error_buy,2)
print("Error",Error)

------MODEL v2--------
All Samples 7

Buy samples:  5
Sum profit b2 buy 3.12
Accuracy 60.00%
Error buy -0.01

Sell samples:  2
Sum profit b2 sell 7.77
Accuracy 100.00%
Error sell 1.51

Overall MODEL v2
Profit 10.89
Accuracy 76.00%
Error 1.5


In [62]:
import pandas as pd

# create example dataframe
df = pd.DataFrame({
    'Time': ['2039-03-30', '2039-03-31', '2039-04-01'],
    'Price': [100, 101, 102]
})

# convert the 'Time' column to datetime format
df['Time'] = pd.to_datetime(df['Time'])

# filter by time > 2039-03-31
filtered_df = df[df['Time'] > '2039-03-31']

print(filtered_df)


        Time  Price
2 2039-04-01    102
